# Packages

In [514]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime
from decimal import Decimal, ROUND_DOWN

# Data & Functions

## Today's DateTime

In [515]:
current_dt = datetime.now() # current date and time
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"] # list of months

## Format Amount

Amounts for each item should be properly formatted. There are two functions for this, but they are processed differently due to analytical reasons.

- `truncate_amount()`: Eliminates values that are placed more than 2 times after the decimal points. However, it must return as a <u>float</u> since the **Revenue** column from the DataFrame contains floats, not strings.

- `format_amount()`: Formats the amount properly and returns it as a <u>string</u>. 

In [516]:
def truncate_amount(amount: float) -> Decimal:
    """
    Truncates decimal places with displaying only two numbers after the decimal point

    Args:
        amount (float): item amount

    Returns:
        Decimal: item amount with exactly two numbers after the decimal point
    """
    return Decimal(amount).quantize(Decimal('0.00'), rounding=ROUND_DOWN)

In [517]:
def format_amount(amount: float) -> str:
    """
    Formats amount to a string by adding '$'
    followed by the amount with the option to
    add 0's after the decimal point.

    Args:
        amount (float): item amount

    Returns:
        str: stringified amount
    """
    return f'${amount:.2f}'

# DataFrames

## OrderDataFrame

In [518]:
class OrderDataFrame(pd.DataFrame):
    
    def __init__(self, data):
        super().__init__(data)
        self.__order_count = 0 # current orders taken (doesn't decrease)
        
        
        
    @property
    def _constructor(self):
        return OrderDataFrame
    
    
    
    def add_item_to_order(self, customer: str, id: int, item: str, quantity: str):
        """
        Adds item to the current order number 

        Args:
            customer (str): name of the customer
            id (int): ID of the item
            item (str): name of the item
            quantity (str): quantity of the item to add
        """
        self.loc[len(self)] = [self.__order_count, customer, id, item, quantity]
    
    
    
    def remove_item_from_order(self, order: int, id: int, quantity: int):
        """
        Removes item(s) from the order. If item quantity reaches 0, the item order 
        gets removed from the DataFrame.

        Args:
            order (int): customer order number
            id (int): cafe item ID
            quantity (int): quantity of the customer cafe item for removal

        Raises:
            ValueError: Invalid order number and/or item ID
            ValueError: Quantity for item removal is greater than current item quantity
        """
        filtered = (self['Order Number'] == order) & (self['Item ID'] == id) # filtered statement
        order_info = self[filtered] # gets order info by filtered statement
        
        if(len(order_info) == 0):
            raise ValueError('Order not found. Check for correct order number and item ID.')
                
        if(quantity > order_info['Quantity'].iloc[0]):
            raise ValueError('Quantity for item removal is greater than current item quantity')
            
        self.loc[self.index[filtered].tolist()[0], 'Quantity'] -= quantity # deducts customer item quantity 
        if(self.loc[self.index[filtered].tolist()[0], 'Quantity'] == 0): # removes customer item info due to zero quantity
            self.drop(self[filtered].index[0], inplace=True)
    
    
    
    def remove_order(self, order_indices: list[int]):
        """
        Removes customer's order (item rows of the same order number)

        Args:
            order_indices (list[int]): list of indices with the same order number

        Raises:
            ValueError: Order number is not found
        """        
        self.drop(order_indices, inplace=True)
    
    
    
    def increase_order_count(self):
        """Increases order count"""
        self.__order_count += 1
        
        
    
    def get_order_count(self) -> int:
        """
        Gets order count

        Returns:
            int: order count
        """
        return self.__order_count

In [519]:
order_df = OrderDataFrame(data=pd.DataFrame(columns=['Order Number',  'Customer', 'Item ID', 'Item','Quantity']))

## CafeDataFrame

In [520]:
class CafeDataFrame(pd.DataFrame):
    
    
    def __init__(self, data: pd.DataFrame, brand: str=None, location: str=None):
        """
        Creates an instance of a cafe dataframe

        Args:
            data (pd.DataFrame): data for DataFrame
            brand (str, optional): Name of the cafe brand. Defaults to None.
            location (str, optional): Location of the cafe. Defaults to None.
        """
        super().__init__(data)
        self.__brand = brand
        self.__location = location
    
    
    
    @property
    def _constructor(self):
        return CafeDataFrame
        
     
        
    def summarize(self):
        """
        Gets summary of today's cafe statement
        
        The summary includes the following:
            - Total amount of items left and sold
            - Total revenue including item revenues
        """
        print('-------STATEMENT--------')
        print(f'Name: {self.__brand}')
        print(f'Address: {self.__location}')
        print(f'Date: {months[current_dt.month-1]} {current_dt.day}, {current_dt.year}\n')
        print(f'Total Orders Made: {order_df.get_order_count()}')
        print(f'Total Items Sold: {self['Sold Count'].sum()}')
        print(f'Total Items Left: {self['Available'].sum()}')
        print('------------------------')
                
        for _, row in self[self['Revenue'] > 0].iterrows():
            print(f'{row['Item']}: {format_amount(truncate_amount(row['Revenue']))}')
        
        print(f'\nTotal Revenue: {format_amount(self.__sum_revenue())}')    

 
 
    def purchase(self, customer: str, items: list[dict[int,int]], order_df: OrderDataFrame):
        """        
        Performs the following for each cafe item:
            1. Increase order count
            2. Decrease Availability count for each item
            3. Increase Sold Count for each item
            4. Increase Revenue for each item
            5. Insert each item to the order

        Args:
            customer (str): name of the customer
            items (list[dict[int,int]]): list of cafe items (ids & quantities) to purchase
            order_df (OrderDataFrame): corresponding OrderDataFrame of the cafe

        Raises:
            ValueError: Item is sold out
            ValueError: Item quantity is higher than availability
        """
        
        order_df.increase_order_count() # new order number
        
        for item in items:
            if self.loc[item['Item ID'],'Sold Out'] == True:
                raise ValueError('SOLD OUT')
            
            if item['Quantity'] > self.loc[item['Item ID'],'Available']:
                raise ValueError('Quantity value is higher than availability')
            
            self.loc[item['Item ID'],'Available'] -= item['Quantity'] # decreases availability count
            self.loc[item['Item ID'],'Sold Count'] += item['Quantity'] # increases sold count
            self.loc[item['Item ID'],'Revenue'] += truncate_amount(self.loc[item['Item ID'],'Price'] * item['Quantity']) # increases revenue
            
            if self.loc[item['Item ID'],'Available'] == 0:
                self.loc[item['Item ID'],'Sold Out'] = True # item now sold out
    
            order_df.add_item_to_order(customer, item['Item ID'], self.loc[item['Item ID'],'Item'], item['Quantity']) # adds item to the order



    def refund(self, order_df: OrderDataFrame, order: int, items: list[dict[int,int]]):
        """
        Performs the following for each cafe item:
            1. Increases Availability count
            2. Decreases Sold count
            3. Sets Sold Out to True
            4. Issues refund to customer
            5. Removes item(s) or order 

        Args:
            order_df (OrderDataFrame): corresponding OrderDataFrame
            order (int): customer's order number
            items (list[dict[int,int]]): items and their quantities to remove for refunding
            
        Raises:
            ValueError: Item ID is negative
            ValueError: Quantity value is 0 or negative
        """
        for item in items:
            
            if(item['Item ID'] < 0):
                raise ValueError(f'Item ID cannot be negative. Got {item['Item ID']}')
            
            if(item['Quantity'] <= 0):
                raise ValueError(f'Quantity value cannot be 0 or negative. Got {item['Quantity']}')
            
            self.loc[item['Item ID'],'Available'] += item['Quantity'] # increases availability count
            self.loc[item['Item ID'],'Sold Count'] -= item['Quantity'] # decreases sold count
            self.loc[item['Item ID'],'Sold Out'] = False # sets False to indicate that the item is now not sold out
            self.loc[item['Item ID'],'Revenue'] -= truncate_amount(self.loc[item['Item ID'],'Price'] * item['Quantity']) # decreases revenue
            order_df.remove_item_from_order(order, item['Item ID'], item['Quantity'])
   


    def refund_all(self, order_df: OrderDataFrame, order: int):
        """
        Refunds all customer cafe items. See `CafeDataFrame.refund()` for more information

        Args:
            order_df (OrderDataFrame): corresponding OrderDataFrame
            order (int): customer's order number

        Raises:
            ValueError: Order number is not found
        """
        order_items = order_df[order_df['Order Number'] == order] # list of items with the same order number
        
        if(len(order_items) == 0):
            raise ValueError(f'Order number {order} is not found')
        
        for _, item in order_items.iterrows(): # loops through all customer order items from the same order number
            self.loc[item['Item ID'],'Available'] += item['Quantity'] # increases availability count
            self.loc[item['Item ID'],'Sold Count'] -= item['Quantity'] # decreases sold count
            self.loc[item['Item ID'],'Sold Out'] = False # sets False to indicate that the item is now not sold out
            self.loc[item['Item ID'],'Revenue'] -= truncate_amount(self.loc[item['Item ID'],'Price'] * item['Quantity']) # decreases revenue
        
        order_df.remove_order(order_items.index)
   
   
   
    def __sum_revenue(self) -> float:
        """
        Gets total revenue from all items sold

        Returns:
            float: Total revenue from all items sold
        """
        total = 0
        for _, row in self.iterrows():
            total += row["Revenue"]
        return truncate_amount(total)

In [521]:
cafe_df = CafeDataFrame(
    data=pd.read_json('./items.json'), 
    brand='Ninja Cafe', 
    location='40711 Field Street Fremont, CA, USA'
)

cafe_df["Sold Count"] = 0 # amount of items sold so far
cafe_df["Sold Out"] = False # flag to detect sold out items
# cafe_df["Revenue"] = 0.00 # income made for selling a specified item
cafe_df['Revenue'] = Decimal(0).quantize(Decimal('0.00'), rounding=ROUND_DOWN)

In [522]:
cafe_df

,Item,Type,Price,Available,Sold Count,Sold Out,Revenue
0,Coffee,Drink,1.99,80,0,False,0.00
1,Croissant,Bakery,2.49,50,0,False,0.00
2,Cheese Danish,Bakery,2.69,40,0,False,0.00
3,"Bacon, Egg & Cheese",Sandwich,5.49,20,0,False,0.00
4,Almond Croissant,Bakery,2.59,30,0,False,0.00
5,Latte,Drink,3.09,40,0,False,0.00


# Happy Hour

Some or all cafe items are configured by happy hour times and discount ptgs.

In [523]:
hh_discount = 0.30 # happy hour discount
hh_start, hh_end = 14, 16 # happy hour start and end hours using 24HR format
hh_type = None # item type for apply for discount (set None to disable happy hour)

In [524]:
def apply_discount(row: pd.Series) -> float:
    """
    Applies discount to cafe items

    Args:
        row: cafe item row

    Returns:
        float: discounted price
    """
    if hh_type is not None:
        if (hh_type == "All" or row["Type"] == hh_type) and (current_dt.hour >= hh_start and current_dt.hour <= hh_end):
            row["Price"] -= row["Price"] * hh_discount # applies discount
            return truncate_amount(row["Price"])
    
    return row["Price"]

In [525]:
cafe_df['Price'] = cafe_df.apply(apply_discount, axis=1)
cafe_df

,Item,Type,Price,Available,Sold Count,Sold Out,Revenue
0,Coffee,Drink,1.99,80,0,False,0.00
1,Croissant,Bakery,2.49,50,0,False,0.00
2,Cheese Danish,Bakery,2.69,40,0,False,0.00
3,"Bacon, Egg & Cheese",Sandwich,5.49,20,0,False,0.00
4,Almond Croissant,Bakery,2.59,30,0,False,0.00
5,Latte,Drink,3.09,40,0,False,0.00


# Transactions

## Purchases

In [526]:
cafe_df.purchase('Ivan',[
    {'Item ID': 0, 'Quantity': 5}, # item ID and its quantity to add
    {'Item ID': 1, 'Quantity': 8},
    {'Item ID': 2, 'Quantity': 3},
    {'Item ID': 3, 'Quantity': 10},
], order_df)


cafe_df.purchase('Vince',[
    {'Item ID': 0, 'Quantity': 20},
    {'Item ID': 5, 'Quantity': 1},
], order_df)


cafe_df.purchase('Jones',[
    {'Item ID': 0, 'Quantity': 1},
    {'Item ID': 3, 'Quantity': 1},
    {'Item ID': 5, 'Quantity': 12},
], order_df)

In [527]:
order_df

,Order Number,Customer,Item ID,Item,Quantity
0,1,Ivan,0,Coffee,5
1,1,Ivan,1,Croissant,8
2,1,Ivan,2,Cheese Danish,3
3,1,Ivan,3,"Bacon, Egg & Cheese",10
4,2,Vince,0,Coffee,20
5,2,Vince,5,Latte,1
6,3,Jones,0,Coffee,1
7,3,Jones,3,"Bacon, Egg & Cheese",1
8,3,Jones,5,Latte,12


In [528]:
cafe_df

,Item,Type,Price,Available,Sold Count,Sold Out,Revenue
0,Coffee,Drink,1.99,54,26,False,51.71
1,Croissant,Bakery,2.49,42,8,False,19.92
2,Cheese Danish,Bakery,2.69,37,3,False,8.07
3,"Bacon, Egg & Cheese",Sandwich,5.49,9,11,False,60.39
4,Almond Croissant,Bakery,2.59,30,0,False,0.00
5,Latte,Drink,3.09,27,13,False,40.15


## Refunds

### Each Item

In [529]:
cafe_df.refund(order_df, order=1, 
    items=[
        {'Item ID': 1, 'Quantity': 4},
        {'Item ID': 3, 'Quantity': 10}, # row removed
    ]
)

cafe_df.refund(order_df, order=2, 
    items=[
        {'Item ID': 0, 'Quantity': 12},
    ]
)

In [530]:
order_df

,Order Number,Customer,Item ID,Item,Quantity
0,1,Ivan,0,Coffee,5
1,1,Ivan,1,Croissant,4
2,1,Ivan,2,Cheese Danish,3
4,2,Vince,0,Coffee,8
5,2,Vince,5,Latte,1
6,3,Jones,0,Coffee,1
7,3,Jones,3,"Bacon, Egg & Cheese",1
8,3,Jones,5,Latte,12


In [531]:
cafe_df

,Item,Type,Price,Available,Sold Count,Sold Out,Revenue
0,Coffee,Drink,1.99,66,14,False,27.84
1,Croissant,Bakery,2.49,46,4,False,9.96
2,Cheese Danish,Bakery,2.69,37,3,False,8.07
3,"Bacon, Egg & Cheese",Sandwich,5.49,19,1,False,5.49
4,Almond Croissant,Bakery,2.59,30,0,False,0.00
5,Latte,Drink,3.09,27,13,False,40.15


### All Items

In [532]:
cafe_df.refund_all(order_df, order=3) # refunds all customer items

In [533]:
order_df

,Order Number,Customer,Item ID,Item,Quantity
0,1,Ivan,0,Coffee,5
1,1,Ivan,1,Croissant,4
2,1,Ivan,2,Cheese Danish,3
4,2,Vince,0,Coffee,8
5,2,Vince,5,Latte,1


In [534]:
cafe_df

,Item,Type,Price,Available,Sold Count,Sold Out,Revenue
0,Coffee,Drink,1.99,67,13,False,25.86
1,Croissant,Bakery,2.49,46,4,False,9.96
2,Cheese Danish,Bakery,2.69,37,3,False,8.07
3,"Bacon, Egg & Cheese",Sandwich,5.49,20,0,False,0.00
4,Almond Croissant,Bakery,2.59,30,0,False,0.00
5,Latte,Drink,3.09,39,1,False,3.08


# Today's Summary

In [535]:
cafe_df.summarize()

-------STATEMENT--------
Name: Ninja Cafe
Address: 40711 Field Street Fremont, CA, USA
Date: October 17, 2025

Total Orders Made: 3
Total Items Sold: 21
Total Items Left: 239
------------------------
Coffee: $25.86
Croissant: $9.96
Cheese Danish: $8.07
Latte: $3.08

Total Revenue: $46.97
